In [1]:
from __future__ import print_function
import sys
sys.path.append('../build/')
%pylab inline
np.set_printoptions(precision=4, suppress=True)
import versor as vsr
import hyperdual
import motor_jacobian

Populating the interactive namespace from numpy and matplotlib


In [2]:
rotor = (vsr.Biv(1,0,0).unit() * pi/6).exp()
rotor1 = vsr.Rot(1,0,0,0)
a = vsr.Vec(1,0,0)
b = a.spin(rotor)

In [3]:
# hyperdual.hyperdual1(a,b,rotor)

In [4]:
# hyperdual.hyperdual2(a,b,rotor)

In [5]:
 def jacrot(a, rot):
    j = np.zeros((3,3))
    for i in range(3):
        biv = vsr.Biv(0.0, 0.0, 0.0)
        biv[i] = 1.0
        j[:,i] = 2.0 * np.array(biv.comm(a.spin(rot))) # Wrong sign?
    return j

In [6]:
J = jacrot(a,rotor1)
print(J)

[[ 0.  0.  0.]
 [-2.  0.  0.]
 [ 0. -2.  0.]]


In [7]:
a ^ b * 2

Biv: [ 1.732 0 0 ]

In [8]:
res, grad, H = hyperdual.hyperdualB(a, b, rotor1) 
print(grad)
print()
print(H)

[ 1.7321  0.      0.    ]

[[ 3.      0.      0.    ]
 [ 0.      3.      1.7321]
 [ 0.      1.7321  1.    ]]


In [9]:
vectors_a = [vsr.Vec(1,0,0), vsr.Vec(0,1,0), vsr.Vec(0,0,1)]
vectors_b = [a.spin(rotor) for a in vectors_a]

In [10]:
n_points=10
vectors_a = [vsr.Vec(*np.random.normal(0.0, 0.8, 3)) for i in range(n_points)]
vectors_b = [vector.spin(rotor) for vector in vectors_a]

In [11]:
def update(rot):
    ans = [hyperdual.hyperdualB(a,b,rot) for a,b in zip(vectors_a, vectors_b)]
    res = np.sum([t[0] for t in ans])
    grad = np.sum([t[1] for t in ans], axis=0)
    Hess = np.sum([t[2] for t in ans], axis=0)
    B = np.dot(np.linalg.pinv(Hess),grad)
    rot = vsr.Biv(*B).exp() * rot
    grad_norm = np.linalg.norm(grad)
    return rot, res, grad_norm

In [12]:
rot = vsr.Rot(1,0,0,0)
for i in range(3):
    rot, res, grad_norm = update(rot)
    print(res)
print(rot)

8.17753306669
0.0944158864625
8.78592360236e-08
Rot: [ 0.87 -0.5 2.2e-18 -3.1e-17 ]


In [13]:
rot = vsr.Rot(1,0,0,0)
J = np.array([jacrot(a,rot) for a in vectors_a]).reshape(-1,3)
print(np.dot(J.T,J))

[[ 65.4203 -11.0731   5.3403]
 [-11.0731  64.1451   1.8383]
 [  5.3403   1.8383  44.7531]]


In [14]:
Hess = np.sum([hyperdual.hyperdualB(a,b,rot)[2] for a,b in zip(vectors_a, vectors_b)], axis=0)
print(Hess)

[[ 49.0652 -10.6172  -0.7896]
 [-10.6172  57.7051   9.3161]
 [ -0.7896   9.3161  51.1931]]


In [15]:
motor1 = vsr.Mot(1,0,0,0,0,0,0,0)
motor = vsr.Vec(1,1,1).trs() * vsr.Rot(vsr.Biv(0,1,0) * np.pi/6.0)
points_a = [a.null() for a in vectors_a]
points_b = [a.spin(motor) for a in points_a]
a = points_a[0]
b = points_b[0]

In [16]:
motor0 = vsr.Mot(1,0,0,0,0,0,0,0)
r,g,H = hyperdual.hyperdualmotor(a,b, motor0)
# print(g)
print(H)

[[ 0.463  -0.7886  0.5791 -1.7956 -0.2651  0.    ]
 [-0.7886  0.7801 -1.9809  0.9857  0.     -0.2651]
 [ 0.5791 -1.9809  4.0351  0.      0.9857  1.7956]
 [-1.7956  0.9857  0.      4.      0.      0.    ]
 [-0.2651  0.      0.9857  0.      4.      0.    ]
 [ 0.     -0.2651  1.7956  0.      0.      4.    ]]


In [17]:
ans = [hyperdual.hyperdualmotor(a,b,motor0) for a,b in zip(points_a, points_b)]
res = np.sum([t[0] for t in ans])
grad = np.sum([t[1] for t in ans], axis=0)
Hess = np.sum([t[2] for t in ans], axis=0)

In [18]:
print(grad)

[-16.8065  34.9785  23.2384  32.7579  20.      21.8544]


In [19]:
print(Hess)

[[ 79.7794 -11.7864  10.4699 -35.3965  23.2119   0.    ]
 [-11.7864  23.6617   9.0987   2.0974   0.      23.2119]
 [ 10.4699   9.0987  45.7905   0.       2.0974  35.3965]
 [-35.3965   2.0974   0.      40.       0.       0.    ]
 [ 23.2119   0.       2.0974   0.      40.       0.    ]
 [  0.      23.2119  35.3965   0.       0.      40.    ]]


In [20]:
def update(rot):
    ans = [hyperdual.hyperdualmotor(a,b,rot) for a,b in zip(points_a, points_b)]
    res = np.sum([t[0] for t in ans])
    grad = np.sum([t[1] for t in ans], axis=0)
    Hess = np.sum([t[2] for t in ans], axis=0)
    B = -np.dot(np.linalg.pinv(Hess),grad)
    rot = vsr.Dll(*B).exp() * rot
    grad_norm = np.linalg.norm(grad)
    return rot, res, grad_norm

In [21]:
rot = vsr.Mot(1,0,0,0,0,0,0,0)
res = 1e6
i = 0
while res > 1e-6:
    i += 1
    rot, res, grad_norm = update(rot)
    print(res)
    print(rot)
print(i)

30.3242872198
Mot: [ 0.68 -0.051 -0.59 -0.44 -0.08 0.11 0.71 0.094 ]
13.6474440534
Mot: [ 0.91 -0.038 -0.41 -0.0099 -0.78 -0.47 -0.28 -0.19 ]
0.331908566537
Mot: [ 0.87 -0.0031 -0.49 -0.0043 -0.69 -0.43 -0.19 -0.24 ]
0.00160808395415
Mot: [ 0.87 3.4e-07 -0.5 8.3e-07 -0.68 -0.43 -0.18 -0.25 ]
1.83237476235e-11
Mot: [ 0.87 3.4e-07 -0.5 8.3e-07 -0.68 -0.43 -0.18 -0.25 ]
5


In [22]:
print(rot.rev() * motor)

Mot: [ 1 1.2e-07 7.5e-07 -8.9e-07 1.5e-07 -1.6e-07 1.3e-07 0 ]


In [23]:
import motor_jacobian

In [24]:
def jacmot(a,mot):
    j = np.zeros((3,6))
    for i in range(6):
        dll = vsr.Dll(0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
        dll[i] = 1.0
        j[:,i] = 2. * np.array(dll.comm(a.spin(mot)))[:3]
    return j

In [25]:
J = np.array([jacmot(a,motor0) for a in points_a]).reshape(-1,6)

In [26]:
np.dot(J.T,J)

array([[ 65.4203, -11.0731,   5.3403, -15.3965,  -9.5461,   0.    ],
       [-11.0731,  64.1451,   1.8383,  23.9517,   0.    ,  -9.5461],
       [  5.3403,   1.8383,  44.7531,   0.    ,  23.9517,  15.3965],
       [-15.3965,  23.9517,   0.    ,  40.    ,   0.    ,   0.    ],
       [ -9.5461,   0.    ,  23.9517,   0.    ,  40.    ,   0.    ],
       [  0.    ,  -9.5461,  15.3965,   0.    ,   0.    ,  40.    ]])